In [ ]:
# -*- coding: utf-8 -*-
# 爬取 人教版 教材

from bs4 import BeautifulSoup
import requests
import re
import os
from fake_useragent import UserAgent

url = 'https://bp.pep.com.cn/jc' 
path = '/users/apple/downloads/人教版'

ua = UserAgent()
# 请求头
headers = {"User-Agent": ua.random}
# 请求网址
resp = requests.get(url=url, headers=headers)
data = resp.text.encode("latin1").decode("utf-8")

soup = BeautifulSoup(data, 'lxml')
li_list = soup.find_all('li')

for i in li_list:
    dirs = i.a.get('href'),i.a.get_text()
    print(dirs[-1])
    sub_path = path + '/' + dirs[-1][:2] + '教材'
    print(sub_path)

    try:
        os.mkdir(sub_path)
    except:
        pass

    file_path = sub_path + '/' + dirs[-1]
    try:
        os.mkdir(file_path)
    except:
        pass
    
    os.chdir(file_path)
    url_sub = url + i.a.get('href')[1:]
    print(url_sub)
    ua = UserAgent()
    headers = {"User-Agent": ua.random}
    
    resp = requests.get(url=url_sub, headers=headers)
    data = resp.text.encode("latin1").decode("utf-8")

    soup = BeautifulSoup(data, 'lxml')
    sub_li = soup.find_all('li')

    for j in sub_li:
        filename = file_path + '/' + j.h6.text + '.pdf'
        string = j.find('div','con_type_link_jcdzs2020').find_all('a')[-1]
        
        url_ssub = url_sub + string.get('href')[2:]
        
        r = requests.get(url_ssub)
        r.raise_for_status()
        
        f = open(filename,'wb')
        for chunk in r.iter_content(100000):
            f.write(chunk)
        f.close()
